In [1]:
#GA U-Net

In [8]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np


In [ ]:
#Main file-path
PATH = '/content/drive/My Drive/...'

#input data path
INPATH = PATH + '/...'

#target data path
OUPATH = PATH + '/...'

#checkpoints data path
CKPATH = PATH + '/...'

#GCP Console Command
imgurl = !ls -1 '{INPATH}' #all in one column

In [3]:
n = 500
train_n = round(n * 0.80)

#Randomized lists
randurls = np.copy(imgurls)
np.random.shuffle(randurls)

#Train/Test Split
tr_urls = randurls[:train_n]
ts_urls = randurls[train:n]

print(len(imgurl), len(tr_urls), len(ts_urls))

NameError: name 'np' is not defined

In [4]:
#Fixing image size

IMG_WIDTH = 256
IMG_HEIGH = 256

#resizing the images function
def resize(inimg, tgimg, height, width):
    inimg = tf.resize(inimg, [height, width])
    tgimg = tf.resize(tgimg, [height, width])
    
    return inimg, tgimg

#normalize pictures function 
def normalize(inimg, tgimg):
    inimg = (inimg / 127.5) - 1
    tgimg = (inimg / 127.5) - 1
    
    return inimg, tgimg

In [5]:
#Data augmentation function (Crop and Flip)
def random_jitter(inimg, tgimg):
    inimg, tgimg = resize(inimg, tgimg, 286, 286)
    #stacking function
    stacked_img = tf.stack([inimg, tgimg], axis=0)
    cropped_img = tf.image.random_crop(stacked_image, size=[2, IMG_HEIGH, IMG_WIDTH, 3])
    
    inimg, tgimg = cropped_image[0], cropped_image[1]
    
    if tf.random.uniform(()) > 0.5:
        
        inimg = tf.image.flip_left_right(inimg)
        tgimg = tf.image.flip_left_right(tgimg)
        
    return inimg, tgimg

In [6]:
def load_image(filename, augment=True):
    
    inimg, tgimg = resize(inimg, tgimg, IMG_HEIGHT, IMG_WIDTH)
    
    if augment:
        inimg, tgimg = random_jitter(inimg, tgimg)
        
    inimg, tgimg = normalize(inimg, tgimg)
    
    return inimg, tgimg

NameError: name 'true' is not defined

In [7]:
def load_train_image(filename):
    return load_image(filename, True)

def load_test_image(filename):
    return load_image(filename, False)

In [9]:
#Generating training datasets from tensor slices
train_dataset = tf.data.Dataset.from_tensor_slices(tr_urls)
train_dataset = train_dataset.map(load_train_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)
train_dataset = train_dataset.batch(1)

test_dataset = tf.data.Dataset.from_tensor_slices(tr_urls)
test_dataset = test_dataset.map(load_test_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)
test_dataset = test_dataset.batch(1)

TypeError: from_tensor_slices() missing 1 required positional argument: 'tensors'

In [10]:
from tensorflow.keras.layers import *
from tensorflow.keras import Sequential, Model

def downSample(filters):
    
    result = Sequential()
    initializer = tf.random_normal_initializer(0, 0.02)
    
    #Convolutional Layer
    result.add(Conv2D(filters,
                      kernel_size=4,
                      strides=2,
                      padding='same',
                      kernel_initializer=initializer,
                      use_bias=not apply_batchnorm))
    #BatchNorm Layer
    if apply_batchnorm:
        result.add(BatchNormalization())
     
    #Activation
    result.add(LeakyReLU)
    
    return result

SyntaxError: invalid syntax (<ipython-input-10-3f0f2aa0b346>, line 14)

In [11]:
def upsample(filters, apply_dropout=True):
    
    result = Sequential()
    initializer = tf.random_normal_initializer(0, 0.02)
    
    #Convolutional Layer
    result.add(Conv2DTranspose(filters,
                               kernel_size=4,
                               strides=2,
                               padding='same',
                               kernel_initializer=initializer,
                               use_bias=False))
    #BatchNorm Layer
    result.add(BatchNormalization())
    
    if apply_dropout:
        result.add(Dropout(0.5))
     
    #Activation
    result.add(ReLU)
    
    return result

In [ ]:
def generator():
    inputs = tf.keras.layers.Input(shape=[None, None, 3])
    
    down_stack = [
        downsample(64, apply_batchnorm=False),
        downsample(128),
        downsample(256),
        downsample(512),
        downsample(512),
        downsample(512),
        downsample(512),
        downsample(512),
    ]
    
    up_stack = [
        upsample(512, apply_dropout=True),
        upsample(512, apply_dropout=True),
        upsample(512, apply_dropout=True),
        upsample(512),
        upsample(256),
        upsample(128),
        upsample(64),
    ]
    
    initializer = tf.random_normal_initializer(0, 0.02)
    
    last = Conv2DTranspose(filters=3,
                          kernel_size=4,
                          strides=2,
                          padding='same',
                          kernel_initializer = initializer,
                          activation='tanh')
    
    x = inputs
    skip_x = []
    concat = Concatenate()
    
    for down in down_stack:
        x = down(x)
        skip_x.append(x)
    
    skip_x = reversed(skip_x[:-1])
    
    for up, sk in zip(up_stack, skip_x):
        x = up(x)
        x = concat([x, sk])
    
    last = last(x)
    
    return Model(inputs=inputs, outputs=last)

In [ ]:
def Discriminator():
    
    initializer = tf.random_normal_initializer(0, 0.02)
    
    ini = Input(shape=[None, None, 3], name='input_img')
    gen = Input(shape=[None, None, 3], name='gen_img')

    con = concatenate([ini, gen])

    down1 = downsample(64, apply_batchnorm=False)(con)
    down2 = downsample(128, 4)(down1)
    down3 = downsample(256, 4)(down2)
    down4 = downsample(256, 4)(down3)
    
    last = Conv2D(filters=1,
                  kernel_size=4,
                  strides=1,
                  padding='same',
                  kernel_initializer=initializer)(down4)
    
    return Model(input=[ini, gen], outputs=last)

In [12]:
loss_object = tf.keras.losses.BinaryCrossentropy(from_logits=True) #logits True = 0 to 1

def discriminatorLoss(disc_real_output, disc_generated_output):
    #Real true vs detected by discriminator
    real_loss = loss_object(tf.ones_like(disc_real_output), disc_real_output)
    #Real false vs detected by discriminator
    generated_loss = loss_object(tf.zeros_like(disc_generated_output), disc_generated_output)
    
    total_disc_loss = real_loss + generated_loss
    
    return total_disc_loss

SyntaxError: invalid syntax (<ipython-input-12-b744a5d84793>, line 3)

In [13]:
lamda = 100

def generator_loss(disc_generated_output, gen_output, target):
    gan_loss = loss_object(tf.ones_like(disc_generated_output), disc_generated_output)
    #Mean Absolute Error
    l1_loss = tf.reduce_mean(tf.abs(target - gen_output))
    
    total_gen_loss = gan_loss + l1_loss
    
    return total_gen_loss

In [ ]:
import os

generator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
discriminator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)

checkpoint_prefix = os.path.join(CKPATH, 'ckpt')
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                discriminator_optimizer=discriminator_optimizer,
                                generator=generator,
                                discriminator=discriminator)

#checkpoint.restore(tf.train.latest_checkpoint(CKPATH)).assert_consumed()

In [ ]:
def generate_images(model, test_input, tar, save_filename=False, display_imgs=True):
    
    prediction = model(test_input, training=True)
    
    if save_filename:
        tf.keras.preprocessing.image.save_img(PATH + '/output/' + save_filename + '.jpg', prediction[0, ...])
        
    plt.figure(figsize=(10,10))

    display_list = [test_input[0], tar[0], prediction[0]]
    title = ['Input Image', 'Ground Truth', 'Predicted Image']
    
    if display_imgs:
        for i in range(3):
            plt.subplot(1, 3, i+1)
            plt.title(title[i])
            # getting the pixel values between [0, 1] to plot it.
            plt.imshow(display_list[i] * 0.5 + 0.5)
            plt.axis('off')
    plt.show()

In [ ]:
@tf.function()
def train_step(input_image, target):
    
    with tf.GradientTape() as gen_tape, tf.GradientTape as discr_tape:
    
        output_image = generator(input_image, training=Ture)
        output_gen_discr = discriminator([output_image, input_image], training=True)
        output_trg_discr = discriminator([target, input_image], training=True)
        discr_loss = discriminator_loss(output_trg_discr, output_gen_discr)
        gen_loss = generator_loss(output_gen_discr, output_image, target)
        
        generator_grads = gen_tape.gradient(gen_loss, generator.trainable_variables)
        discriminator_grads = discr_tape.gradient(discr_loss, discriminator.trainable_variables)
        generator_optimizer.apply_gradients(zip(generator_grads, generator.trainable_variables))
        discriminator_optimizer.apply_gradients(zip(discriminator_grads, discriminator.trainable_variables))

In [ ]:
from IPython.display import clear_output

def train(dataset, epochs):
    for epoch in range(epochs):
        imgi = 0
        for input_image, target in dataset:
            print('Epoch:'  + str(epoch) + ' - Train: ' + str(imgi) + '/' + str(len(tr_urls)))
            imgi += 1
            train_step(input_image, target)
        
            clear_outputs(wait=True)
        
        for inp, tar in test_dataset.take(5):
            generate_images(generator, inp, tar, str(imgi) + '_' + str(epoch), display_imgs=True)
            
        if (epoch + 1) % 25 == 0:
            checkpoint.save(file_prefix = checkpoint_prefix)

In [ ]:
#train(train_dataset, 100)